<a href="https://colab.research.google.com/github/park1NG/BigDataSecurity/blob/main/TeamProject_MalwareDetection2_SIFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 드라이브 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') 

In [ ]:
cd '/content/drive/Shareddrives/BigDataSecurity'

In [ ]:
ls

## 모듈 불러오기

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils, datasets
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torch.utils.data
import os
import time
# 전처리
import cv2
import numpy as np
from skimage.feature import hog
import matplotlib.pyplot as plt

In [ ]:
from PIL import ImageFile
from PIL import Image
ImageFile.LOAD_TRUNCATED_IMAGES = True # prevent truncate error

## 데이터셋 불러오기


In [ ]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, ConcatDataset

class CustomDataset(Dataset):
    def __init__(self, root, malware_list, transform=None):
        self.root = root
        self.malware_list = malware_list
        self.transform = transform
        self.data = []
        self.targets = []
        self.folder_to_label = {}  # 폴더명과 클래스 레이블 매핑을 위한 딕셔너리
        self.class_counts = {}  # 클래스별 이미지 개수를 저장하는 딕셔너리

        for idx, folder_name in enumerate(os.listdir(root)):
            if folder_name in malware_list:
                folder_path = os.path.join(root, folder_name)
                self.folder_to_label[folder_name] = self.label_transform(folder_name)  # 폴더명에 대한 레이블 변환 결과 저장

                count = 0  # 클래스별 이미지 개수 초기화
                for image_name in os.listdir(folder_path):
                    if image_name.endswith('.png'):  # 이미지 파일 확장자 지정
                        image_path = os.path.join(folder_path, image_name)
                        self.data.append(image_path)
                        self.targets.append(folder_name)
                        count += 1  # 클래스별 이미지 개수 증가

                self.class_counts[folder_name] = count

    def __getitem__(self, index):
        image_path = self.data[index]
        target = self.targets[index]
        image = Image.open(image_path)

        if self.transform is not None:
            image = self.transform(image)

        target = self.folder_to_label[target]  # 폴더명에 대한 레이블 변환 결과 가져오기
        return image, target, image_path

    def __len__(self):
        return len(self.data)

    def label_transform(self, label):
        # 예시: 폴더명을 기준으로 클래스 레이블 변환
        if label != 'Other':
            return 0
        else:
            return 1

def concat_datasets(datasets):
    return ConcatDataset(datasets)


In [ ]:
# 이미지의 RGB 채널별 통계량 확인 함수
def print_stats(dataset):
    imgs = np.array([img.numpy() for img, _, _ in dataset])
    print(f'shape: {imgs.shape}')
    
    min_r = np.min(imgs, axis=(2, 3))[:, 0].min()
    min_g = np.min(imgs, axis=(2, 3))[:, 1].min()
    min_b = np.min(imgs, axis=(2, 3))[:, 2].min()

    max_r = np.max(imgs, axis=(2, 3))[:, 0].max()
    max_g = np.max(imgs, axis=(2, 3))[:, 1].max()
    max_b = np.max(imgs, axis=(2, 3))[:, 2].max()

    mean_r = np.mean(imgs, axis=(2, 3))[:, 0].mean()
    mean_g = np.mean(imgs, axis=(2, 3))[:, 1].mean()
    mean_b = np.mean(imgs, axis=(2, 3))[:, 2].mean()

    std_r = np.std(imgs, axis=(2, 3))[:, 0].std()
    std_g = np.std(imgs, axis=(2, 3))[:, 1].std()
    std_b = np.std(imgs, axis=(2, 3))[:, 2].std()
    
    print(f'min: {min_r, min_g, min_b}')
    print(f'max: {max_r, max_g, max_b}')
    print(f'mean: {mean_r, mean_g, mean_b}')
    print(f'std: {std_r, std_g, std_b}')

In [ ]:
pre_transforms = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),        
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

In [ ]:
image_transforms1 = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),        
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

image_transforms2 = {
    "train": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),        
    ]),
    "test": transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
}

In [ ]:
malware_list = {'Adposhel': 0,
 'Allaple': 0,
 'Amonetize': 0,
 'Autorun': 0,
 'Other': 1,
 'BrowseFox': 0,
 'Dinwod': 0,
 'InstallCore': 0,
 'MultiPlug': 0,
 'VBA': 0,
 'Vilsel': 0}

In [ ]:
def label_transform(label):
    # 예시: 클래스 레이블을 원하는 형태로 변환
    if label != 4:
      return 0
    else:
      return 1

In [ ]:
train_path1 = './malware/train'
test_path1 = './malware/val'
train_path2 = './malware2/train'
test_path2 = './malware2/val'

In [ ]:
train_data1 = CustomDataset(root = train_path1, malware_list=malware_list,
                                  transform = pre_transforms['train'])
train_data2 = CustomDataset(root = train_path2, malware_list=malware_list,
                                  transform = pre_transforms['train'])

test_data1 = CustomDataset(root = test_path1, malware_list=malware_list,
                                  transform = pre_transforms['test'])
test_data2 = CustomDataset(root = test_path2, malware_list=malware_list,
                                  transform = pre_transforms['test'])

In [ ]:
# Dataset rebalance 
test_data, add_data = torch.utils.data.random_split(test_data2, [2714, 220])

train_data = ConcatDataset([train_data1, train_data2, test_data1, add_data])

In [ ]:
print_stats(train_data)

In [ ]:
print_stats(test_data)

In [ ]:
train_data1 = CustomDataset(root = train_path1, malware_list=malware_list,
                                  transform = image_transforms1['train'])
train_data2 = CustomDataset(root = train_path2, malware_list=malware_list,
                                  transform = image_transforms2['train'])

test_data1 = CustomDataset(root = test_path1, malware_list=malware_list,
                                  transform = image_transforms1['test'])
test_data2 = CustomDataset(root = test_path2, malware_list=malware_list,
                                  transform = image_transforms2['test'])

In [ ]:
# Total : 13,566
# Train : 10852
# Test : 2714(1357 * 2)

In [ ]:
print(len(train_data1))
print(len(train_data2))
print(len(test_data1))
print(len(test_data2))

In [ ]:
# Dataset rebalance 
test_data, add_data = torch.utils.data.random_split(test_data2, [2714, 220])

train_data = ConcatDataset([train_data1, train_data2, test_data1, add_data])

In [ ]:
def TestLoader(Dataset):
  val_size = int(0.5 * len(Dataset))
  test_size = int(0.5 * len(Dataset))

  valid_data, test_data = torch.utils.data.random_split(Dataset, [val_size, test_size])

  valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=128, shuffle=False) # make test loader
  test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False) # make test loader

  return valid_loader, test_loader

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True) # make train loader
valid_loader, test_loader = TestLoader(test_data) # make valid & test loader

In [ ]:
def show_datasize():
  print(f'Number of training examples: {len(train_data)}')
  print(f'Number of validation examples: {int(len(test_data) / 2)}')
  print(f'Number of testing examples: {int(len(test_data) / 2)}')

show_datasize()

In [ ]:
train_data.class_to_idx  = malware_list # class name

In [ ]:
classes = train_data.class_to_idx
classes

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # functions to show an image


# def imshow(img):
#     npimg = img.numpy()
#     plt.imshow(np.transpose(npimg, (1, 2, 0)))
#     plt.show()


# # get some random training images
# dataiter = iter(train_loader)
# # print(dataiter.next())
# images, labels = next(dataiter)

# batch_size = 16

# # show images
# imshow(torchvision.utils.make_grid(images))
# # print labels
# print()

# labels = labels.tolist()
# print(' '.join(f'{list(classes.keys())[list(classes.values()).index(j)]}' for j in labels))

## 전처리

### Laplacian Filtering (Edge 강화) && Bilateral Filtering (노이즈 제거) 적용

In [ ]:
import os
import cv2
import numpy as np

# Train 폴더 위치
train_folder = './malware/train/'

# Bila_Train 폴더 위치
bila_folder = '/home/Bila_folder1'

# Laplace_Bila_Train 폴더 위치
both_folder = '/home/both_folder1'

# 모든 클래스에 대해 불러옵니다.
for class_folder in os.walk(train_folder).__next__()[1]:
    # 클래스 이름을 불러옵니다.
    class_name = class_folder
    # 클래스 폴더의 경로를 붙여넣습니다.
    class_folder_path = os.path.join(train_folder, class_folder) 
    # Bila_Train 폴더에 저장할 경로를 지정하고 생성합니다.
    bila_class_folder_path = os.path.join(bila_folder, class_folder)
    # Both_Train 폴더에 저장할 경로를 지정하고 생성합니다.
    both_class_folder_path = os.path.join(both_folder, class_folder)
    os.makedirs(both_class_folder_path, exist_ok=True)

    # 클래스 폴더를 순회하며 Laplacian Filtering과 Bilateral Filtering을 적용한 이미지를 저장합니다.
    for image_file in os.listdir(class_folder_path):
        # 파일 경로를 붙여넣습니다.
        image_file_path = os.path.join(class_folder_path, image_file)

        # 이미지 파일을 불러옵니다.
        image = cv2.imread(image_file_path)

        # 이미지가 제대로 불러와지지 않은 경우 다음 이미지로 넘어갑니다.
        if image is None:
            continue

        # Laplacian Filtering을 적용합니다.
        lap_image = cv2.Laplacian(image, cv2.CV_8U, ksize=3)

        # Bilateral Filtering을 적용합니다.
        bila_image = cv2.bilateralFilter(image, -1, 10, 5)

        # Laplace_Bila_Train 폴더에 라플라시안 이미지를 저장합니다.
        lap_image_path = os.path.join(both_class_folder_path, 'lap_' + image_file)
        cv2.imwrite(lap_image_path, lap_image)

        # Laplace_Bila_Train 폴더에 Bilateral 이미지를 저장합니다.
        bila_image_path = os.path.join(both_class_folder_path, 'bila_' + image_file)
        cv2.imwrite(bila_image_path, bila_image)

        # 파일 경로를 붙여넣습니다.
        both_image_path = os.path.join(both_class_folder_path, 'both_' + image_file)

        # both 이미지가 이미 저장된 경우 lap_image와 bila_image의 기존 이미지를 제거하고 both_image를 저장합니다.
        if os.path.isfile(both_image_path):
            os.remove(lap_image_path)
            os.remove(bila_image_path)

        # 이미지를 불러와서 Laplacian Filtering과 Bilateral Filtering을 모두 적용합니다.
        lap_image = cv2.imread(lap_image_path)
        bila_image = cv2.imread(bila_image_path)

        # 이미지가 제대로 불러와지지 않은 경우 다음 이미지로 넘어갑니다.
        if lap_image is None or bila_image is None:
            continue

        both_image = cv2.addWeighted(lap_image, 0.5, bila_image, 0.5, 0)

        # Laplace_Bila_Train 폴더에 Laplacian Filtering과 Bilateral Filtering을 모두 적용한 이미지를 저장합니다.
        cv2.imwrite(both_image_path, both_image)

In [ ]:
import os
import cv2
import numpy as np

# Train 폴더 위치
train_folder = './malware2/train/'

# Bila_Train 폴더 위치
bila_folder = '/home/Bila_folder2'

# Laplace_Bila_Train 폴더 위치
both_folder = '/home/both_folder2'

# 모든 클래스에 대해 불러옵니다.
for class_folder in os.walk(train_folder).__next__()[1]:
    # 클래스 이름을 불러옵니다.
    class_name = class_folder
    # 클래스 폴더의 경로를 붙여넣습니다.
    class_folder_path = os.path.join(train_folder, class_folder) 
    # Bila_Train 폴더에 저장할 경로를 지정하고 생성합니다.
    bila_class_folder_path = os.path.join(bila_folder, class_folder)
    # Both_Train 폴더에 저장할 경로를 지정하고 생성합니다.
    both_class_folder_path = os.path.join(both_folder, class_folder)
    os.makedirs(both_class_folder_path, exist_ok=True)

    # 클래스 폴더를 순회하며 Laplacian Filtering과 Bilateral Filtering을 적용한 이미지를 저장합니다.
    for image_file in os.listdir(class_folder_path):
        # 파일 경로를 붙여넣습니다.
        image_file_path = os.path.join(class_folder_path, image_file)

        # 이미지 파일을 불러옵니다.
        image = cv2.imread(image_file_path)

        # 이미지가 제대로 불러와지지 않은 경우 다음 이미지로 넘어갑니다.
        if image is None:
            continue

        # Laplacian Filtering을 적용합니다.
        lap_image = cv2.Laplacian(image, cv2.CV_8U, ksize=3)

        # Bilateral Filtering을 적용합니다.
        bila_image = cv2.bilateralFilter(image, -1, 10, 5)

        # Laplace_Bila_Train 폴더에 라플라시안 이미지를 저장합니다.
        lap_image_path = os.path.join(both_class_folder_path, 'lap_' + image_file)
        cv2.imwrite(lap_image_path, lap_image)

        # Laplace_Bila_Train 폴더에 Bilateral 이미지를 저장합니다.
        bila_image_path = os.path.join(both_class_folder_path, 'bila_' + image_file)
        cv2.imwrite(bila_image_path, bila_image)

        # 파일 경로를 붙여넣습니다.
        both_image_path = os.path.join(both_class_folder_path, 'both_' + image_file)

        # both 이미지가 이미 저장된 경우 lap_image와 bila_image의 기존 이미지를 제거하고 both_image를 저장합니다.
        if os.path.isfile(both_image_path):
            os.remove(lap_image_path)
            os.remove(bila_image_path)

        # 이미지를 불러와서 Laplacian Filtering과 Bilateral Filtering을 모두 적용합니다.
        lap_image = cv2.imread(lap_image_path)
        bila_image = cv2.imread(bila_image_path)

        # 이미지가 제대로 불러와지지 않은 경우 다음 이미지로 넘어갑니다.
        if lap_image is None or bila_image is None:
            continue

        both_image = cv2.addWeighted(lap_image, 0.5, bila_image, 0.5, 0)

        # Laplace_Bila_Train 폴더에 Laplacian Filtering과 Bilateral Filtering을 모두 적용한 이미지를 저장합니다.
        cv2.imwrite(both_image_path, both_image)

### HOG

In [ ]:
hog_features = []
hog = cv2.HOGDescriptor()

In [ ]:
pip install scikit-image

In [ ]:
import cv2
import numpy as np
from skimage.util import view_as_blocks
from sklearn.preprocessing import StandardScaler

In [ ]:
# Gabor 필터 생성 함수
def create_filters(scales, orientations):
    filters = []
    for scale in range(scales[0], scales[1] + 1):
        for orientation in np.arange(0, np.pi, np.pi / orientations):
            filt_real = cv2.getGaborKernel((scale, scale), 1, orientation, scale, 0, ktype=cv2.CV_32F)
            filt_imag = cv2.getGaborKernel((scale, scale), 1, orientation, scale, 0.5 * np.pi, ktype=cv2.CV_32F)
            filt = filt_real + filt_imag
            filt /= 2.0 * np.pi * scale * scale
            filters.append(filt)
    return filters

# HOG 디스크립터 계산 함수
def hog_descriptor_single_channel(image, scales=(8, 8), orientations=8, blocks=(4, 4)):    # Gabor 필터 생성
    filters = create_filters(scales, orientations)
    
    # 이미지 크기와 블록 크기 계산
    height, width = image.shape[:2]
    block_size = height // blocks[0], width // blocks[1]

    padding_size = blocks[0] * block_size[0] - height, blocks[1] * block_size[1] - width
    
    # 이미지 패딩 (필요한 경우)
    if padding_size != (0, 0):
        image = cv2.copyMakeBorder(image, 0, padding_size[0], 0, padding_size[1], cv2.BORDER_CONSTANT, value=0)
    
    # 이미지를 블록으로 분할
    block_shape = (block_size[0], block_size[1])
    blocks = view_as_blocks(image, block_shape=(block_size[0], block_size[1])).reshape(-1, *block_size, order='F')
    
    # 각 블록의 GIST 특성 추출
    features = []
    for block in blocks:
        feats = []
        for scale in filters:
            for filt in scale:
                filtered = cv2.filter2D(block, cv2.CV_64F, filt)
                feats.append(filtered.mean())
        features.append(feats)
    
    # 전체 GIST 디스크립터로 결합
    return np.concatenate(features)


def hog_descriptor(image, scales=(8, 8), orientations=8, blocks=(4, 4)):
    if len(image.shape) == 3:
    # 각 채널에 대해 GIST 디스크립터 계산
        descriptors = [hog_descriptor_single_channel(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), scales, orientations, blocks)]            # 전체 GIST 디스크립터로 결합
        return np.concatenate(descriptors)
    else:
    # 단일 채널 이미지의 경우 GIST 디스크립터를 한 번만 계산
        return hog_descriptor_single_channel(image, scales, orientations, blocks)

In [ ]:
# 각 이미지에 맞는 레이블 생성
def load_labels(dir, num_samples):
    y = []
    for subdir in sorted(os.listdir(dir)):
        subdir_path = os.path.join(dir, subdir)
        if os.path.isdir(subdir_path):
            for i, filename in enumerate(sorted(os.listdir(subdir_path))):
                y.append(labels_dict[subdir])
                if len(y) == num_samples:
                    break
        if len(y) == num_samples:
            break
    return np.array(y)

In [ ]:
# 각 악성코드 이미지 폴더에서 350개의 이미지에 대한 gist descriptor를 계산하여 반환
def get_hog_descriptors(train_data):
    descriptors = []
    labels = []
    for i, v in enumerate(train_data):
      _,label,path = v
      # 파일 경로 생성
      # 이미지 로드
      image = cv2.imread(path)
      # 이미지에 대한 GIST 디스크립터 계산
      descriptor = hog_descriptor(image)
      descriptors.append(descriptor)
      labels.append(label)
      if i % 100 == 99:
        print("\tProcessed", i + 1, "images")
    return np.array(descriptors) , np.array(labels)

In [ ]:
descriptors, train_labels = get_hog_descriptors(train_data)
print(len(descriptors))
print('HOG Descriptor Shape:', descriptors.shape)

In [ ]:
pip install scikit-learn xgboost

In [ ]:
# 데이터와 레이블 설정
X = descriptors
hog_features = X

# hog_descriptors 및 레이블 불러오기
X_train = X
X_val , val_label= get_hog_descriptors(add_data)
X_test, test_label = get_hog_descriptors(test_data)

In [ ]:

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 정의
models = {
    'Random Forest': make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100)),
    'XGBoost': XGBClassifier(learning_rate=0.01, reg_lambda=0.1),
    'Linear SVM': make_pipeline(StandardScaler(), LinearSVC(penalty='l2', dual=False)),
    'SMO': make_pipeline(StandardScaler(), SVC(kernel='rbf', C=1.0, gamma='scale', probability=True)),
    'J48': DecisionTreeClassifier()
}

# 성능 지표
scores = {'accuracy': accuracy_score, 'FPR': confusion_matrix, 'precision': precision_score, 'recall': recall_score, 'f1score': f1_score}

# 각 모델에 대해 교차 검증 및 테스트 세트에서 성능 평가
for model_name, model_instance in models.items():
    print(model_name)
    model_instance.fit(X_train_scaled, train_labels)
    y_pred = model_instance.predict(X_test_scaled)
    for score_name, score_func in scores.items():
        if score_name == 'FPR':
            cm = score_func(test_label, y_pred)
            fp = cm.sum(axis=0) - np.diag(cm)
            tn = cm.sum() - (cm.sum(axis=1) + fp)
            fpr = np.mean(fp / (fp + tn))
            print(score_name, fpr)
        elif score_name == 'accuracy':
            print(score_name, score_func(test_label, y_pred))
        else:
            print(score_name, score_func(test_label, y_pred, average='weighted'))

###GIST

In [ ]:
pip install scikit-image

In [ ]:
import cv2
import numpy as np
from skimage.util import view_as_blocks

In [ ]:
# Gabor 필터 생성 함수
def create_filters(scales, orientations):
    filters = []
    for scale in range(scales[0], scales[1] + 1):
        for orientation in np.arange(0, np.pi, np.pi / orientations):
            filt_real = cv2.getGaborKernel((scale, scale), 1, orientation, scale, 0, ktype=cv2.CV_32F)
            filt_imag = cv2.getGaborKernel((scale, scale), 1, orientation, scale, 0.5 * np.pi, ktype=cv2.CV_32F)
            filt = filt_real + filt_imag
            filt /= 2.0 * np.pi * scale * scale
            filters.append(filt)
    return filters

# GIST 디스크립터 계산 함수
def gist_descriptor_single_channel(image, scales=(8, 8), orientations=8, blocks=(4, 4)):    # Gabor 필터 생성
    filters = create_filters(scales, orientations)
    
    # 이미지 크기와 블록 크기 계산
    height, width = image.shape[:2]
    block_size = height // blocks[0], width // blocks[1]

    padding_size = blocks[0] * block_size[0] - height, blocks[1] * block_size[1] - width
    
    # 이미지 패딩 (필요한 경우)
    if padding_size != (0, 0):
        image = cv2.copyMakeBorder(image, 0, padding_size[0], 0, padding_size[1], cv2.BORDER_CONSTANT, value=0)
    
    # 이미지를 블록으로 분할
    block_shape = (block_size[0], block_size[1])
    blocks = view_as_blocks(image, block_shape=(block_size[0], block_size[1])).reshape(-1, *block_size, order='F')
    
    # 각 블록의 GIST 특성 추출
    features = []
    for block in blocks:
        feats = []
        for scale in filters:
            for filt in scale:
                filtered = cv2.filter2D(block, cv2.CV_64F, filt)
                feats.append(filtered.mean())
        features.append(feats)
    
    # 전체 GIST 디스크립터로 결합
    return np.concatenate(features)


def gist_descriptor(image, scales=(8, 8), orientations=8, blocks=(4, 4)):
    if len(image.shape) == 3:
    # 각 채널에 대해 GIST 디스크립터 계산
        descriptors = [gist_descriptor_single_channel(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), scales, orientations, blocks)]            # 전체 GIST 디스크립터로 결합
        return np.concatenate(descriptors)
    else:
    # 단일 채널 이미지의 경우 GIST 디스크립터를 한 번만 계산
        return gist_descriptor_single_channel(image, scales, orientations, blocks)

In [ ]:
cd 

In [ ]:
# 각 악성코드 이미지 폴더에서 350개의 이미지에 대한 gist descriptor를 계산하여 반환
def get_gist_descriptors(root_dir):
    descriptors = []
    for subdir in sorted(os.listdir(root_dir)):
        subdir_path = os.path.join(root_dir, subdir)
        if os.path.isdir(subdir_path):
            print("Processing directory:", subdir_path)
            for i, filename in enumerate(os.listdir(subdir_path)):
                # 파일 경로 생성
                filepath = os.path.join(subdir_path, filename)
                # 이미지 로드
                image = cv2.imread(filepath)
                # 이미지에 대한 GIST 디스크립터 계산
                descriptor = gist_descriptor(image)
                descriptors.append(descriptor)
                
                if i % 10 == 9:
                    print("\tProcessed", i + 1, "images")
    return np.array(descriptors)


In [ ]:
root_dir = "Train/"
# 각 폴더에서 350개의 이미지에 대한 gist descriptor 계산
descriptors = get_gist_descriptors(root_dir)
print(len(descriptors))
print('GIST Descriptor Shape:', descriptors.shape)

In [ ]:
GIST_descriptors = descriptors

###SIFT

In [ ]:
sift_features = []
sift = cv2.xfeatures2d.SIFT_create()

In [ ]:
# Gabor 필터 생성 함수
def create_filters(scales, orientations):
    filters = []
    for scale in range(scales[0], scales[1] + 1):
        for orientation in np.arange(0, np.pi, np.pi / orientations):
            filt_real = cv2.getGaborKernel((scale, scale), 1, orientation, scale, 0, ktype=cv2.CV_32F)
            filt_imag = cv2.getGaborKernel((scale, scale), 1, orientation, scale, 0.5 * np.pi, ktype=cv2.CV_32F)
            filt = filt_real + filt_imag
            filt /= 2.0 * np.pi * scale * scale
            filters.append(filt)
    return filters

# SIFT 디스크립터 계산 함수
def sift_descriptor_single_channel(image, scales=(8, 8), orientations=8, blocks=(4, 4)):    # Gabor 필터 생성
    filters = create_filters(scales, orientations)
    
    # 이미지 크기와 블록 크기 계산
    height, width = image.shape[:2]
    block_size = height // blocks[0], width // blocks[1]

    padding_size = blocks[0] * block_size[0] - height, blocks[1] * block_size[1] - width
    
    # 이미지 패딩 (필요한 경우)
    if padding_size != (0, 0):
        image = cv2.copyMakeBorder(image, 0, padding_size[0], 0, padding_size[1], cv2.BORDER_CONSTANT, value=0)
    
    # 이미지를 블록으로 분할
    block_shape = (block_size[0], block_size[1])
    blocks = view_as_blocks(image, block_shape=(block_size[0], block_size[1])).reshape(-1, *block_size, order='F')
    
    # 각 블록의 GIST 특성 추출
    features = []
    for block in blocks:
        feats = []
        for scale in filters:
            for filt in scale:
                filtered = cv2.filter2D(block, cv2.CV_64F, filt)
                feats.append(filtered.mean())
        features.append(feats)
    
    # 전체 GIST 디스크립터로 결합
    return np.concatenate(features)

def sift_descriptor(image, scales=(8, 8), orientations=8, blocks=(4, 4)):
    if len(image.shape) == 3:
    # 각 채널에 대해 GIST 디스크립터 계산
        descriptors = [sift_descriptor_single_channel(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), scales, orientations, blocks)]            # 전체 GIST 디스크립터로 결합
        return np.concatenate(descriptors)
    else:
    # 단일 채널 이미지의 경우 GIST 디스크립터를 한 번만 계산
        return sift_descriptor_single_channel(image, scales, orientations, blocks)

In [ ]:
# 각 악성코드 이미지 폴더에서 350개의 이미지에 대한 gist descriptor를 계산하여 반환
def get_sift_descriptors(train_data):
    descriptors = []
    labels = []
    for i, v in enumerate(train_data):
      _,label,path = v
      # 파일 경로 생성
      # 이미지 로드
      image = cv2.imread(path)
      # 이미지에 대한 GIST 디스크립터 계산
      descriptor = sift_descriptor(image)
      descriptors.append(descriptor)
      labels.append(label)
      if i % 100 == 99:
        print("\tProcessed", i + 1, "images")
    return np.array(descriptors) , np.array(labels)

In [ ]:
descriptors, train_labels = get_sift_descriptors(train_data)
print(len(descriptors))
print('SIFT Descriptor Shape:', descriptors.shape)

In [ ]:
# 데이터와 레이블 설정
X = descriptors

# hog_descriptors 및 레이블 불러오기
X_train = X
X_val , val_label= get_sift_descriptors(add_data)
X_test, test_label = get_sift_descriptors(test_data)

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from xgboost import XGBClassifier


In [ ]:
# 레이블 설정
labels_dict = classes  

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 정의
models = {
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'Linear SVM': LinearSVC(),
    'SMO': SVC(kernel='rbf'), # SMO는 일반적으로 서포트 벡터 머신 (SVM)이 rbf 커널을 사용
    'J48': DecisionTreeClassifier() # J48는 scikit-learn에서 Decision Tree에 해당
}

# 성능 지표
scores = {'accuracy': accuracy_score, 'FPR': confusion_matrix, 'precision': precision_score, 'recall': recall_score, 'f1score': f1_score}

# 각 모델에 대해 교차 검증 및 테스트 세트에서 성능 평가
for model_name, model_instance in models.items():
    print(model_name)
    model_instance.fit(X_train_scaled, train_labels)
    y_pred = model_instance.predict(X_test_scaled)
    for score_name, score_func in scores.items():
        if score_name == 'FPR':
            cm = score_func(test_label, y_pred)
            fp = cm.sum(axis=0) - np.diag(cm)
            tn = cm.sum() - (cm.sum(axis=1) + fp)
            fpr = np.mean(fp / (fp + tn))
            print(score_name, fpr)
        elif score_name == 'accuracy':
            print(score_name, score_func(test_label, y_pred))
        else:
            print(score_name, score_func(test_label, y_pred, average='weighted'))

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import matplotlib.pyplot as plt

models = [
    ('XGBoost', xgb.XGBClassifier()),
    ('Linear SVM', LinearSVC()),
    ('SMO', SVC(kernel='poly', coef0=1.0, C=1.0, degree=3)),
    ('Random Forest', RandomForestClassifier(n_estimators=100)),
    ('J48', DecisionTreeClassifier(max_depth=6))
]

def root_mean_squared_error(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    return rmse

def run_cross_validation(X_train, train_labels, models, n_splits=5):
    scores = {model_name: {'Validation Accuracy': [], 'Validation Loss': []} for model_name, _ in models}
    
    cv = StratifiedKFold(n_splits = n_splits)
    for train_index, val_index in cv.split(X_train, train_labels):
        X_cv_train, X_cv_val = X_train[train_index], X_train[val_index]
        train_labels = np.array(train_labels)  # 이 줄을 추가하여 train_labels를 numpy 배열로 변환합니다.
        y_cv_train, y_cv_val = train_labels[train_index], train_labels[val_index]
    
        for model_name, model_instance in models:
            print(f"Processing {model_name}...")
            sc = StandardScaler()
            X_cv_train_std = sc.fit_transform(X_cv_train)
            model = model_instance
            model.fit(X_cv_train_std, y_cv_train)
    
            X_cv_val_std = sc.transform(X_cv_val)
            y_val_pred = model.predict(X_cv_val_std)
            scores[model_name]['Validation Accuracy'].append(accuracy_score(y_cv_val, y_val_pred))
            scores[model_name]['Validation Loss'].append(root_mean_squared_error(y_cv_val, y_val_pred))
    
    return scores

scores_per_model = run_cross_validation(X_train, train_labels, models)

# Plot validation accuracy per model
n_splits = 5  # or any positive integer value
plt.figure(figsize=(10, 6))
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
for model_name, model_scores in scores_per_model.items():
    plt.plot(range(1, n_splits + 1), model_scores['Validation Accuracy'], label=model_name)
plt.legend()
plt.show()

# Plot validation loss per model
plt.figure(figsize=(10, 6))
plt.xlabel('Epochs')
plt.ylabel('Validation Loss')
for model_name, model_scores in scores_per_model.items():
    plt.plot(range(1, n_splits + 1), model_scores['Validation Loss'], label=model_name)
plt.legend()
plt.show()


##EDA

###Train_data에 대한 클래스별 데이터 분포(pi graph)

In [ ]:
# Count the number of images for each class
image_counts = [train_data.class_counts[malware_name] for malware_name in classes]

# Plot a pie chart
plt.pie(image_counts, labels=classes, autopct='%1.1f%%')
plt.show()

###val_data에 대한 클래스별 데이터 분포(pi graph)

In [ ]:
import os
import matplotlib.pyplot as plt

# Count the number of images for each class in add_data
add_data_counts = [add_data.targets.count(malware_name) for malware_name in classes]

# Plot a pie chart for add_data
plt.figure(figsize=(8, 8))
plt.pie(add_data_counts, labels=classes, autopct='%1.1f%%')
plt.title("Class Distribution for add_data")
plt.show()

# Count the number of images for each class in val_data
test_data_counts = [test_data.targets.count(malware_name) for malware_name in classes]

# Plot a pie chart for val_data
plt.figure(figsize=(8, 8))
plt.pie(test_data_counts, labels=classes, autopct='%1.1f%%')
plt.title("Class Distribution for val_data")
plt.show()


### 모델 불러오기

In [ ]:
from sklearn.manifold import TSNE # sklearn 사용하면 easy !! 
import numpy as np
from matplotlib import pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

시각화에 사용할 모델 불러오기

In [ ]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x
model = torchvision.models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features # fc의 입력 노드 수를 산출한다. 512개
model.fc = nn.Linear(num_ftrs, 10) # fc를 nn.Linear(num_ftrs, 10)로 대체, CIFAR10,,
model = model.to(device)

In [ ]:
# classifier 들어가기 직전에 값을 뽑아낼 것임
# (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
print(model)

###T-SNE

기본 데이터 T-SNE

In [ ]:
actual = []
deep_features = []

# Define a color map for each class
color_map = plt.cm.get_cmap('tab10', len(malware_list))

model.eval() # Set the model to evaluation mode
with torch.no_grad():
    for data in train_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features = model(images)

        deep_features += features.cpu().numpy().tolist()
        actual += labels.cpu().numpy().tolist()

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

tsne = TSNE(n_components=4, n_neighbors=55, min_dist=1, metric='manhattan')
cluster = np.array(tsne.fit_transform(np.array(deep_features)))
actual = np.array(actual)

plt.figure(figsize=(10, 10))

def color_map(index):
    # Define your color map logic here
    colors = ['red', 'blue']  # Example color map
    return colors[index]

for label, malware_type in malware_list.items():
    malware_type_arr = np.array([malware_type] * len(actual))  # Convert malware_type to a NumPy array
    idx = np.where(actual == malware_type_arr)
    plt.scatter(cluster[idx, 0], cluster[idx, 1], marker='.', label=label, color=color_map(label))

plt.legend()
plt.show()

## 모델 구성

## Machine Learning

with GIST

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from xgboost import XGBClassifier


In [ ]:
# 데이터와 레이블 설정
X = descriptors

# 레이블 기록
labels_dict = classes
train_dir = "Train/"  # 이 경로에 train 데이터가 저장
val_dir = "Validation/"  # 이 경로에 test 데이터가 저장

# GIST_descriptors 및 레이블 불러오기
X_train = X
X_test = get_gist_descriptors(val_dir)

# 레이블 설정
labels_dict = classes

# 각 이미지에 맞는 레이블 생성
def load_labels(dir):
    y = []
    for subdir in sorted(os.listdir(dir)):
        subdir_path = os.path.join(dir, subdir)
        if os.path.isdir(subdir_path):
            for i, filename in enumerate(sorted(os.listdir(subdir_path))):
                y.append(labels_dict[subdir])
    return np.array(y)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 모델 정의
models = {
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'Linear SVM': LinearSVC(),
    'SMO': SVC(kernel='rbf'), # SMO는 일반적으로 서포트 벡터 머신 (SVM)이 rbf 커널을 사용
    'J48': DecisionTreeClassifier() # J48는 scikit-learn에서 Decision Tree에 해당
}

# 성능 지표
scores = {'accuracy': accuracy_score, 'FPR': confusion_matrix, 'precision': precision_score, 'recall': recall_score, 'f1score': f1_score}

# 각 모델에 대해 교차 검증 및 테스트 세트에서 성능 평가
for model_name, model_instance in models.items():
    print(model_name)
    model_instance.fit(X_train_scaled, y_train)
    y_pred = model_instance.predict(X_test_scaled)
    for score_name, score_func in scores.items():
        if score_name == 'FPR':
            cm = score_func(y_test, y_pred)
            fp = cm.sum(axis=0) - np.diag(cm)
            tn = cm.sum() - (cm.sum(axis=1) + fp)
            fpr = np.mean(fp / (fp + tn))
            print(score_name, fpr)
        elif score_name == 'accuracy':
            print(score_name, score_func(y_test, y_pred))
        else:
            print(score_name, score_func(y_test, y_pred, average='weighted'))


with GIST+HOG

In [ ]:
hog_gist_concatenated = np.concatenate((HOG_descriptors, GIST_descriptors), axis=1)
hog_gist_concatenated.shape

In [ ]:

# 데이터와 레이블 설정
X = hog_gist_concatenated

# 레이블 기록
labels_dict = classes
train_dir = "Train/"  # 이 경로에 train 데이터가 저장
val_dir = "Validation/"  # 이 경로에 test 데이터가 저장

# GIST_descriptors 및 레이블 불러오기
X_train = X
X_test = get_gist_descriptors(val_dir)

# 레이블 설정
labels_dict = classes

# 각 이미지에 맞는 레이블 생성
def load_labels(dir):
    y = []
    for subdir in sorted(os.listdir(dir)):
        subdir_path = os.path.join(dir, subdir)
        if os.path.isdir(subdir_path):
            for i, filename in enumerate(sorted(os.listdir(subdir_path))):
                y.append(labels_dict[subdir])
    return np.array(y)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Length of X:", len(X))
print("Length of y:", len(y))

# 모델 정의
models = {
    'Random Forest': RandomForestClassifier(),
    'XGBoost': XGBClassifier(),
    'Linear SVM': LinearSVC(),
    'SMO': SVC(kernel='rbf'), # SMO는 일반적으로 서포트 벡터 머신 (SVM)이 rbf 커널을 사용
    'J48': DecisionTreeClassifier() # J48는 scikit-learn에서 Decision Tree에 해당
}

# 성능 지표
scores = {'accuracy': accuracy_score, 'FPR': confusion_matrix, 'precision': precision_score, 'recall': recall_score, 'f1score': f1_score}

# 각 모델에 대해 교차 검증 및 테스트 세트에서 성능 평가
for model_name, model_instance in models.items():
    print(model_name)
    model_instance.fit(X_train_scaled, y_train)
    y_pred = model_instance.predict(X_test_scaled)
    for score_name, score_func in scores.items():
        if score_name == 'FPR':
            cm = score_func(y_test, y_pred)
            fp = cm.sum(axis=0) - np.diag(cm)
            tn = cm.sum() - (cm.sum(axis=1) + fp)
            fpr = np.mean(fp / (fp + tn))
            print(score_name, fpr)
        elif score_name == 'accuracy':
            print(score_name, score_func(y_test, y_pred))
        else:
            print(score_name, score_func(y_test, y_pred, average='weighted'))


## Deep Learning

### Model List

#### Model-DNN

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchsummary import summary

In [ ]:
# 모델 정의
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()
        # 모델 구조를 정의합니다.
        self.fc1 = nn.Linear(224 * 224 * 3, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = x.view(x.size(0), -1)  # 입력 데이터를 1차원으로 평탄화합니다.
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.sigmoid(x)  # 이진 분류를 위해 sigmoid 활성화 함수를 적용합니다.
        return x

# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 입력 데이터 생성
x = torch.randn(3, 3, 224, 224).to(device)

# 모델 생성 및 출력 계산
model = DNN().to(device)
output = model(x)
print(output.size())

# 모델 요약 출력
summary(model, (3, 224, 224))


학습

학습 평가

#### Model-CNN basic

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        ############### Conv2d, MaxPool2d, Linear 함수에 들어갈 파라미터를 채우세요 ##############
        self.conv1 = nn.Conv2d(3, 10, 3) # in_channel, out_channel, kernel size
        self.pool = nn.MaxPool2d(3, 2) # kernel_size, stride
        self.conv2 = nn.Conv2d(10, 20, 3)
        self.fc1 = nn.Linear(56180, 160) # in_features, out_features
        self.fc2 = nn.Linear(160, 120)
        self.fc3 = nn.Linear(120, 1)
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.3)
        ###########################################################################################


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

In [ ]:
# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 입력 데이터 생성
x = torch.randn(3, 3, 224, 224).to(device)

# 모델 생성 및 출력 계산
model = Net().to(device)
output = model(x)
print(output.size())

# 모델 요약 출력
summary(model, (3, 224, 224))

#### Model-ResNet50

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, config, output_dim):
        super().__init__()
                
        block, n_blocks, channels = config
        self.in_channels = channels[0]
            
        assert len(n_blocks) == len(channels) == 4
        
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size = 7, stride = 2, padding = 3, bias = False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace = True)
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        
        self.layer1 = self.get_model_layer(block, n_blocks[0], channels[0])
        self.layer2 = self.get_model_layer(block, n_blocks[1], channels[1], stride = 2)
        self.layer3 = self.get_model_layer(block, n_blocks[2], channels[2], stride = 2)
        self.layer4 = self.get_model_layer(block, n_blocks[3], channels[3], stride = 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(self.in_channels, output_dim)
        
    def get_model_layer(self, block, n_blocks, channels, stride = 1):
    
        layers = []
        
        if self.in_channels != block.expansion * channels:
            downsample = True
        else:
            downsample = False
        
        layers.append(block(self.in_channels, channels, stride, downsample))
        
        for i in range(1, n_blocks):
            layers.append(block(block.expansion * channels, channels))

        self.in_channels = block.expansion * channels
            
        return nn.Sequential(*layers)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        h = x.view(x.shape[0], -1)
        x = self.fc(h)
        x = self.sigmoid(x)
        
        return x, h

In [ ]:
class BasicBlock(nn.Module):
    
    expansion = 1
    
    def __init__(self, in_channels, out_channels, stride = 1, downsample = False):
        super().__init__()
                
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 3, 
                               stride = stride, padding = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, 
                               stride = 1, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.relu = nn.ReLU(inplace = True)
        
        if downsample:
            conv = nn.Conv2d(in_channels, out_channels, kernel_size = 1, 
                             stride = stride, bias = False)
            bn = nn.BatchNorm2d(out_channels)
            downsample = nn.Sequential(conv, bn)
        else:
            downsample = None
        
        self.downsample = downsample
        
    def forward(self, x):
        
        i = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        
        if self.downsample is not None:
            i = self.downsample(i)
                        
        x += i
        x = self.relu(x)
        
        return x

In [ ]:
class Bottleneck(nn.Module):
    
    expansion = 4
    
    def __init__(self, in_channels, out_channels, stride = 1, downsample = False):
        super().__init__()
    
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size = 1, 
                               stride = 1, bias = False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size = 3, 
                               stride = stride, padding = 1, bias = False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        self.conv3 = nn.Conv2d(out_channels, self.expansion * out_channels, kernel_size = 1,
                               stride = 1, bias = False)
        self.bn3 = nn.BatchNorm2d(self.expansion * out_channels)
        
        self.relu = nn.ReLU(inplace = True)
        
        if downsample:
            conv = nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size = 1, 
                             stride = stride, bias = False)
            bn = nn.BatchNorm2d(self.expansion * out_channels)
            downsample = nn.Sequential(conv, bn)
        else:
            downsample = None
            
        self.downsample = downsample
        
    def forward(self, x):
        
        i = x
        
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        
        x = self.conv3(x)
        x = self.bn3(x)
                
        if self.downsample is not None:
            i = self.downsample(i)
            
        x += i
        x = self.relu(x)
    
        return x

##### model_info

In [ ]:
from collections import namedtuple
Model_Info = namedtuple('Model_Info', ['block', 'n_blocks', 'channels'])

Model_config = Model_Info(block = Bottleneck,
                               n_blocks = [3, 4, 6, 3],
                               channels = [64, 128, 256, 512])

In [ ]:
import torchvision.models as models
pretrained_model = models.resnet50(pretrained = True)

# Fine Tuning
IN_FEATURES = pretrained_model.fc.in_features 
OUTPUT_DIM = 1
fc = nn.Linear(IN_FEATURES, OUTPUT_DIM)
pretrained_model.fc = fc
model = CustomModel(Model_config, OUTPUT_DIM)

In [ ]:
# Model Load
model.load_state_dict(pretrained_model.state_dict())

In [ ]:
# Count trainable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

### LR_finder

In [ ]:
START_LR = 1e-2

optimizer = optim.Adam(model.parameters(), lr=START_LR)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
from torchsummary import summary
from torchvision import models

x = torch.randn(3, 3, 224, 224).to(device)
output = model(x)

summary(model, (3, 224, 224))

In [ ]:
import torch.optim.lr_scheduler as lr_scheduler
from torch.optim.lr_scheduler import _LRScheduler

class LRFinder:
    def __init__(self, model, optimizer, criterion, device):
        
        self.optimizer = optimizer
        self.model = model
        self.criterion = criterion
        self.device = device
        
        torch.save(model.state_dict(), 'init_params.pt')

    def range_test(self, iterator, end_lr = 10, num_iter = 100, 
                   smooth_f = 0.05, diverge_th = 5):
        
        lrs = []
        losses = []
        best_loss = float('inf')

        lr_scheduler = ExponentialLR(self.optimizer, end_lr, num_iter)
        
        iterator = IteratorWrapper(iterator)
        
        for iteration in range(num_iter):

            loss = self._train_batch(iterator)

            #update lr
            lr_scheduler.step()
            
            lrs.append(lr_scheduler.get_lr()[0])

            if iteration > 0:
                loss = smooth_f * loss + (1 - smooth_f) * losses[-1]
                
            if loss < best_loss:
                best_loss = loss

            losses.append(loss)
            
            if loss > diverge_th * best_loss:
                print("Stopping early, the loss has diverged")
                break
                       
        #reset model to initial parameters
        model.load_state_dict(torch.load('init_params.pt'))
                    
        return lrs, losses

    def _train_batch(self, iterator):
        
        self.model.train()
        
        self.optimizer.zero_grad()
        
        x, y = iterator.get_batch()
        
        x = x.to(self.device)
        y = y.to(self.device)
        
        y_pred, _ = self.model(x)
                
        loss = self.criterion(y_pred, y)
        
        loss.backward()
        
        self.optimizer.step()
        
        return loss.item()

class ExponentialLR(_LRScheduler):
    def __init__(self, optimizer, end_lr, num_iter, last_epoch=-1):
        self.end_lr = end_lr
        self.num_iter = num_iter
        super(ExponentialLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        curr_iter = self.last_epoch + 1
        r = curr_iter / self.num_iter
        return [base_lr * (self.end_lr / base_lr) ** r for base_lr in self.base_lrs]

class IteratorWrapper:
    def __init__(self, iterator):
        self.iterator = iterator
        self._iterator = iter(iterator)

    def __next__(self):
        try:
            inputs, labels = next(self._iterator)
        except StopIteration:
            self._iterator = iter(self.iterator)
            inputs, labels, *_ = next(self._iterator)

        return inputs, labels

    def get_batch(self):
        return next(self)

In [ ]:
END_LR = 10
NUM_ITER = 50

lr_finder = LRFinder(model, optimizer, criterion, device)
lrs, losses = lr_finder.range_test(train_loader, END_LR, NUM_ITER)

In [ ]:
def plot_lr_finder(lrs, losses, skip_start = 10, skip_end = 30):
    
    if skip_end == 0:
        lrs = lrs[skip_start:]
        losses = losses[skip_start:]
    else:
        lrs = lrs[skip_start:-skip_end]
        losses = losses[skip_start:-skip_end]
    
    fig = plt.figure(figsize = (16,8))
    ax = fig.add_subplot(1,1,1)
    ax.plot(lrs, losses)
    ax.set_xscale('log')
    ax.set_xlabel('Learning rate')
    ax.set_ylabel('Loss')
    ax.grid(True, 'both', 'x')
    plt.show()

In [ ]:
FOUND_LR = 1e-2

params = [
          {'params': model.conv1.parameters(), 'lr': FOUND_LR / 10},
          {'params': model.bn1.parameters(), 'lr': FOUND_LR / 10},
          {'params': model.layer1.parameters(), 'lr': FOUND_LR / 8},
          {'params': model.layer2.parameters(), 'lr': FOUND_LR / 6},
          {'params': model.layer3.parameters(), 'lr': FOUND_LR / 4},
          {'params': model.layer4.parameters(), 'lr': FOUND_LR / 2},
          {'params': model.fc.parameters()}
         ]


optimizer = optim.Adam(params, lr = FOUND_LR, weight_decay=0.0001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [ ]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='.init_params.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: '.init_params.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

## 모델 학습

In [ ]:
def train(model, iterator, optimizer, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for (x, y, _) in iterator:
        
        x = x.to(device)
        y = y.to(device)
        
        optimizer.zero_grad()
                
        y_pred = model(x)
        
        loss = criterion(y_pred, y)  
        acc = (y_pred.argmax(dim=1) == y).float().mean()
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    epoch_loss /= len(iterator)
    epoch_acc /= len(iterator)
        
    return epoch_loss, epoch_acc

In [ ]:
def evaluate(model, iterator, criterion, device):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():

        for (x, y, _) in iterator:

           x = x.to(device)
           y = y.to(device)

           y_pred = model(x)
        
           loss = criterion(y_pred, y)  
           acc = (y_pred.argmax(dim=1) == y).float().mean()

           epoch_loss += loss.item()
           epoch_acc += acc.item()
          
    epoch_loss /= len(iterator)
    epoch_acc /= len(iterator)
      
    return epoch_loss, epoch_acc_1

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
best_valid_loss = float('inf')
result_list = []
lr_list = []

patience = 5

early_stopping = EarlyStopping(patience = patience, verbose = True)

for epoch in range(100):
    
    start_time = time.monotonic()
    
    train_loss, train_acc = train(model, train_loader, optimizer, criterion, device)
    valid_loss, valid_acc = evaluate(model, valid_loader, criterion, device)
      
    early_stopping(valid_loss, model)
    lr_list.append(optimizer.param_groups[0]["lr"]) 

    # patience 동안 val_loss가 감소하지 않으면 조기 종료
    if early_stopping.early_stop:
      print("Early stopping")
      break

    # val_loss 감소하면 best model 불러오기 
    model.load_state_dict(torch.load('.init_params.pt'))

    end_time = time.monotonic()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc_1*100:6.2f}%')
    print(f'\tValid Loss: {valid_loss:.3f} | Valid Acc: {valid_acc_1*100:6.2f}%')

    result = {
    'EPOCH': epoch,
    'Train Loss': train_loss,
    'Train acc': train_acc,
    'Valid Loss': valid_loss,
    'Valid acc': valid_acc}
  
    result_list.append(result)
  
result_df = pd.DataFrame(result_list)

## 모델 평가

In [ ]:
# Loss 및 Acc 변화 그래프
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

axes[0].plot(result_df['EPOCH'], result_df['Train Loss'], label='Train Loss')
axes[0].plot(result_df['EPOCH'], result_df['Valid Loss'], label='Valid Loss')
axes[0].legend()
axes[0].set_title('Loss')

axes[1].plot(result_df['EPOCH'], result_df['Train acc'], label='Train acc')
axes[1].plot(result_df['EPOCH'], result_df['Valid acc'], label='Valid acc')
axes[1].legend()
axes[1].set_title('ACC')

# axes[2].plot(result_df['EPOCH'], result_df['Train acc_5'], label='Train acc_5')
# axes[2].plot(result_df['EPOCH'], result_df['Valid acc_5'], label='Valid acc_5')
# axes[2].legend()
# axes[2].set_title('ACC_5')

plt.show()

In [ ]:
plt.plot(lr_list)
plt.xlabel("Epochs")
plt.ylabel("Learning Rate")

In [ ]:
model.load_state_dict(torch.load('.init_params.pt'))

In [ ]:
test_loss, test_acc = evaluate(model, test_loader, criterion, device)

print(f'Test Loss: {test_loss:.3f} | Test Acc @1: {test_acc*100:6.2f}%')

In [ ]:
# 각 항목별 정확도 및 컨퓨전 매트릭스 
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt 
nb_classes = 2

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes) in enumerate(test_loader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs, _ = model(inputs)
        preds = torch.argmax(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1


print('        Malware', '    Benign')
print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [ ]:
# 컨퓨전 매트릭스 heatmap 그리기
import seaborn as sns

ax= plt.subplot()
sns.heatmap(confusion_matrix, annot=True, fmt='g', ax=ax);  #annot=True to annotate cells, ftm='g' to disable scientific notation

# labels, title and ticks
ax.set_xlabel('Predicted labels');ax.set_ylabel('True labels'); 
ax.set_title('Confusion Matrix'); 
ax.xaxis.set_ticklabels(['Malware', 'Benign']); ax.yaxis.set_ticklabels(['Malware', 'Benign']);